# Data Processing in Python using Pandas


---
**author**:
  - **Nader Tavana**

---
 

### Import libraries, reading data and database connection:

In [1]:
import pandas as pd
import numpy as np
import os, os.path
import sqlite3
import tempfile



baza = os.path.join(tempfile.mkdtemp(), 'example.db')
if os.path.isfile(baza):
   os.remove(baza)



Tags = pd.read_csv("travel_stackexchange_com/Tags.csv.gz",compression = "gzip")
Badges = pd.read_csv("travel_stackexchange_com/Badges.csv.gz",compression = "gzip")
Comments = pd.read_csv("travel_stackexchange_com/Comments.csv.gz",compression = "gzip")
Posts = pd.read_csv("travel_stackexchange_com/Posts.csv.gz",compression = "gzip")
PostLinks = pd.read_csv("travel_stackexchange_com/PostLinks.csv.gz",compression = "gzip")
Users = pd.read_csv("travel_stackexchange_com/Users.csv.gz",compression = "gzip")
Votes = pd.read_csv("travel_stackexchange_com/Votes.csv.gz",compression = "gzip")


conn = sqlite3.connect(baza) # create the connection
Badges.to_sql("Badges", conn) # import the data frame into the database
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)


### SQL queries:

### 1)

In [2]:
# query 1
query1=pd.read_sql_query("""
SELECT Posts.Title, RelatedTab.NumLinks
FROM
(SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
FROM PostLinks
GROUP BY RelatedPostId) AS RelatedTab
JOIN Posts ON RelatedTab.PostId=Posts.Id
WHERE Posts.PostTypeId=1
ORDER BY NumLinks DESC

""", conn)


# solution 1 
def solution1():
    
    df=PostLinks.groupby(['RelatedPostId']).size().reset_index(name='NumLinks') #group by RelatedPostId,COUNT(*) 

    RelatedTab=df.rename(columns={"RelatedPostId": "PostId"})    # AS NumLinks               

    df2=pd.merge(RelatedTab,Posts,how='inner',left_on='PostId',right_on='Id') #JOIN Posts ON RelatedTab.PostId=Posts.Id
    
    RelatedTab_Posts=df2[df2['PostTypeId']==1] # WHERE Posts.PostTypeId=1

    RelatedTab_Posts=RelatedTab_Posts.sort_values('NumLinks',ascending=False) #ORDER BY NumLinks DESC

    
    RelatedTab_Posts=RelatedTab_Posts[['Title','NumLinks']] #SELECT Posts.Title, RelatedTab.NumLinks

    RelatedTab_Posts=RelatedTab_Posts.sort_values('Title') # here we have to sort title because some titles have same NumLinks
                                                           # value so for comparison it gives us error
    
    RelatedTab_Posts=RelatedTab_Posts.reset_index(drop=True) # reser index for comparison
    return RelatedTab_Posts

query1=query1.sort_values('Title') # # here we have to sort title because some titles have same NumLinks
                                                           # value so for comparison it gives us error
query1=query1.reset_index(drop=True) # reset index for comparison

solution1=solution1()

print(solution1.equals(query1)) # check equality




True


### 2)


In [5]:
# query 2
query2=pd.read_sql_query("""
SELECT
    Users.DisplayName,
    Users.Age,
    Users.Location,
    SUM(Posts.FavoriteCount) AS FavoriteTotal,
    Posts.Title AS MostFavoriteQuestion,
    MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
FROM Posts
JOIN Users ON Users.Id=Posts.OwnerUserId
WHERE Posts.PostTypeId=1
GROUP BY OwnerUserId
ORDER BY FavoriteTotal DESC
LIMIT 10
""", conn)


# solution 2
def solution2():
    
    # Firstly I join the Users and Posts data frames
    join = pd.merge(Users, Posts[Posts['PostTypeId'] == 1], left_on = 'Id', right_on = 'OwnerUserId')

    # I apply the SUM() and MAX() functions on the merged data frames
    group = join.groupby('OwnerUserId').agg({'FavoriteCount': [np.sum, np.max]}).reset_index()
    group.columns = group.columns.get_level_values(0)
    group.columns = ['OwnerUserId', 'FavoriteTotal', 'FavoriteMax']

    # Since I lost some columns while applying SUM() and MAX() I join the aggregated data frame with merged Posts and Users
    answer = pd.merge(group, join, left_on = ['OwnerUserId', 'FavoriteMax'],
                      right_on = ['OwnerUserId', 'FavoriteCount'], how = 'left')

    # Now I have all the needed columns, I just have to select the right ones and order the rows
    answer = answer[['DisplayName', 'Age', 'Location', 'FavoriteTotal', 'Title', 'FavoriteMax']]
    answer = answer.rename(columns = {'Title': 'MostFavoriteQuestion', 'FavoriteMax': 'MostFavoriteQuestionLikes'})
    answer = answer.sort_values(by = 'FavoriteTotal', ascending = False).reset_index(drop = True).head(10)
    return answer

solution2=solution2()

print(solution2.equals(query2)) # check equality

True


### Comments on 2nd query: 
Firstly calculating the FavoriteTotal and MostFavoriteQuestionLikes. Then we could use column OwnerUserId combined with MostFavoriteQuestion likes to merge the aggregated data with Users and Posts, regaining all the variables lost while aggregating. Then selecting the right columns and order the rows.

### 3)

In [6]:
query3=pd.read_sql_query("""

SELECT
Posts.Title,
CmtTotScr.CommentsTotalScore
FROM (
SELECT
PostID,
UserID,
SUM(Score) AS CommentsTotalScore
FROM Comments
GROUP BY PostID, UserID
) AS CmtTotScr
JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
WHERE Posts.PostTypeId=1
ORDER BY CmtTotScr.CommentsTotalScore DESC
LIMIT 10


""", conn)


def solution3():
    df=Comments.groupby(['PostId','UserId'])['Score'].sum().reset_index() # SUM(Score) AS CommentsTotalScore
    CmtTotScr=df.rename(columns={"Score": "CommentsTotalScore"})          # GROUP BY PostID, UserID
     
    # JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
    Posts_CmtTotScr=pd.merge(CmtTotScr,Posts,how='inner',left_on=['PostId','UserId'],right_on=['Id','OwnerUserId'])

    Posts_CmtTotScr=Posts_CmtTotScr[Posts_CmtTotScr['PostTypeId']==1] # WHERE Posts.PostTypeId=1
    
    Posts_CmtTotScr=Posts_CmtTotScr.sort_values('CommentsTotalScore',ascending=False) # ORDER BY CmtTotScr.CommentsTotalScore DESC
    
    Posts_CmtTotScr=Posts_CmtTotScr[['Title','CommentsTotalScore']] # Posts.Title,CmtTotScr.CommentsTotalScore

    Posts_CmtTotScr=Posts_CmtTotScr.head(10) # LIMIT 10
    
    # We need to sort based on title also because some titles have same CommentsTotalScore hence for comparison we need to sort it
    Posts_CmtTotScr=Posts_CmtTotScr.sort_values(['CommentsTotalScore', 'Title'], ascending=[False, True]) 
    
    Posts_CmtTotScr=Posts_CmtTotScr.reset_index(drop=True) # reset index for comparison
    
    return Posts_CmtTotScr


# # We need to sort based on title also because some titles have same CommentsTotalScore hence for comparison we need to sort it
query3=query3.sort_values(['CommentsTotalScore', 'Title'], ascending=[False, True]) 

query3=query3.reset_index(drop=True) # reset index for comparison

solution3=solution3()

print(solution3.equals(query3)) # check for equality

conn.close()


True
